# For Reading Parameter Files to just see the fit parameter values

In [5]:
%reset -f
import ROOT
import traceback
from MyCommonAnalysisFunction_richcap import *
from array import array
from datetime import datetime


ROOT.TH1.AddDirectory(0)
# ROOT.gStyle.SetTitleOffset(1.3,'y')
ROOT.gStyle.SetTitleOffset(1.75,'y')

ROOT.gStyle.SetGridColor(17)
ROOT.gStyle.SetPadGridX(1)
ROOT.gStyle.SetPadGridY(1)

print(f"{color.BOLD}\nStarting RG-A SIDIS Analysis\n{color.END}")


Starting RG-A SIDIS Analysis



In [66]:
import re
import numpy as np
import pandas as pd

# Regex for ANSI escape sequences
ansi_escape = re.compile(r'\x1B\[[0-?]*[ -/]*[@-~]')

def parse_fit_file(path, nrows=6, ncols=6):
    results = []

    current_q2y = None
    current_zpt = None
    buffer_lines = []
    parB = None
    parC = None
    chi2 = None

    # Regex for header after stripping ANSI
    header_re = re.compile(r"For SMEARED Q2-y Bin (\d+) - z-PT Bin (\d+):")

    with open(path, "r", encoding="utf-8") as f:
        for raw_line in f:
            # Strip ANSI codes
            line = ansi_escape.sub("", raw_line).strip()

            # Match header
            m = header_re.match(line)
            if(m):
                current_q2y = int(m.group(1))
                current_zpt = int(m.group(2))
                buffer_lines = []
                parB = None
                parC = None
                chi2 = None

            elif(line.startswith("Par B")):
                buffer_lines.append(line)
                try:
                    val = line.split("=")[1].split("±")[0].strip()
                    # parB = float(val)
                    # print(f"current_zpt = {current_zpt}\n\tparB = {parB}")
                except Exception:
                    parB = None

            elif(line.startswith("Par C")):
                buffer_lines.append(line)
                try:
                    val = line.split("=")[1].split("±")[0].strip()
                    parC = float(val)
                except Exception:
                    parC = None

            elif(line.startswith("chi2/NDF")):
                buffer_lines.append(line)
                try:
                    val = line.split("=")[1].strip()
                    chi2 = float(val)
                except Exception:
                    chi2 = None

                if((current_q2y is not None) and (current_zpt is not None)):
                    results.append({
                        "Q2y": current_q2y,
                        "zPT": current_zpt,
                        "ParB": parB,
                        "ParC": parC,
                        "Chi2NDF": chi2,
                        "Details": "\n".join(buffer_lines)
                    })
                buffer_lines = []
                parB = None
                parC = None
                chi2 = None

            else:
                if(current_q2y is not None):
                    buffer_lines.append(line)

    df = pd.DataFrame(results)
    if(not df.empty):
        df = df.set_index("zPT").sort_index()

    grid_B = np.full((nrows, ncols), np.nan)
    grid_C = np.full((nrows, ncols), np.nan)
    grid_N = np.full((nrows, ncols), np.nan)

    if(not df.empty):
        for zpt, row in df.iterrows():
            r = (zpt - 1) // ncols
            c = (zpt - 1) % ncols
            if((0 <= r < nrows) and (0 <= c < ncols)):
                grid_B[r, c] = row["ParB"]
                grid_C[r, c] = row["ParC"]
                grid_N[r, c] = row["Chi2NDF"]

    df_B = pd.DataFrame(grid_B, index=[f"Bin {(i*ncols)+1}:" for i in range(nrows)],
                        columns=[f"Bin {j+1}:" for j in range(ncols)])
    df_C = pd.DataFrame(grid_C, index=[f"Bin {(i*ncols)+1}:" for i in range(nrows)],
                        columns=[f"Bin {j+1}:" for j in range(ncols)])
    df_N = pd.DataFrame(grid_N, index=[f"Bin {(i*ncols)+1}:" for i in range(nrows)],
                        columns=[f"Bin {j+1}:" for j in range(ncols)])

    return df, df_B, df_C, df_N


# def plot_grids(df_B, df_C, df_N):
#     plt.imshow(df_B.values, cmap="coolwarm", interpolation="none")
#     plt.colorbar(label="Par B")
#     plt.title("Parameter B across bins (6x6 grid)")
#     plt.show()

#     plt.imshow(df_C.values, cmap="viridis", interpolation="none")
#     plt.colorbar(label="Par C")
#     plt.title("Parameter C across bins (6x6 grid)")
#     plt.show()

#     plt.imshow(df_N.values, cmap="viridis", interpolation="none")
#     plt.colorbar(label="Chi2NDF")
#     plt.title("Chi2NDF across bins (6x6 grid)")
#     plt.show()


import ROOT

def plot_grids(df_B, df_C, df_N):
    nrows, ncols = df_B.shape

    # Histograms
    hB = ROOT.TH2D("ParB", "Parameter B;Column;Row", ncols, 0.5, ncols+0.5, nrows, 0.5, nrows+0.5)
    hC = ROOT.TH2D("ParC", "Parameter C;Column;Row", ncols, 0.5, ncols+0.5, nrows, 0.5, nrows+0.5)
    hN = ROOT.TH2D("Chi2NDF", "Chi2NDF;Column;Row",  ncols, 0.5, ncols+0.5, nrows, 0.5, nrows+0.5)

    # Fill contents (note: DataFrame index is 0-based, ROOT bins are 1-based)
    for r in range(nrows):
        for c in range(ncols):
            b_val = df_B.iat[r, c]
            c_val = df_C.iat[r, c]
            n_val = df_N.iat[r, c]
            if(not pd.isna(b_val)):
                hB.SetBinContent(c+1, r+1, b_val)
            if(not pd.isna(c_val)):
                hC.SetBinContent(c+1, r+1, c_val)
            if(not pd.isna(n_val)):
                hN.SetBinContent(c+1, r+1, n_val)

    c1 = ROOT.TCanvas("c1", "ParB", 1600, 1200)
    c1.Divide(3, 1, 0.001, 0.001)
    c1.cd(1)
    hB.Draw("COLZ TEXT")
    c1.Update()
    c1.cd(2)
    hC.Draw("COLZ TEXT")
    c1.Update()
    c1.cd(3)
    hN.Draw("COLZ TEXT")
    c1.Update()

    return hB, hC, hN, c1


# ----------------------------
# Example usage
# ----------------------------

# 1) From a file:
# df, df_B, df_C = parse_fit_file("fit_parameters.txt", nrows=6, ncols=6, q2y_bin=5)
# print(df.head())
# print("\nGrid of Par B:\n", df_B)
# print("\nGrid of Par C:\n", df_C)

# 2) From an in-memory string (handy for quick tests):
# sample_text = \"\"\"<paste your TXT here>\"\"\"
# df, df_B, df_C = parse_fit_text(sample_text, q2y_bin=5)
# df_B, df_C

# Path to your file
filename = "Parameters_Pass_2_Sector_Tests_FC_14_V1_EvGen_Ran_on_9_22_2025_Q2_y_Bins_5_Smeared.txt"

df, df_B, df_C, df_N = parse_fit_file(filename, nrows=6, ncols=6)
# print(df.head())
print(f"\n{color.BOLD}Grid of Par B:{color.END}\n{df_B}")
print(f"\n{color.BOLD}Grid of Par C:{color.END}\n{df_C}")
# print(f"\n{color.BOLD}Grid of Chi2NDF:{color.END}\n{df_N}")

hB, hC, hN, c1 = plot_grids(df_B, df_C, df_N)

c1.Draw()


Grid of Par B:
         Bin 1:  Bin 2:  Bin 3:  Bin 4:  Bin 5:  Bin 6:
Bin 1:      NaN     NaN     NaN     NaN     NaN     NaN
Bin 7:      NaN     NaN     NaN     NaN     NaN     NaN
Bin 13:     NaN     NaN     NaN     NaN     NaN     NaN
Bin 19:     NaN     NaN     NaN     NaN     NaN     NaN
Bin 25:     NaN     NaN     NaN     NaN     NaN     NaN
Bin 31:     NaN     NaN     NaN     NaN     NaN     NaN

Grid of Par C:
               Bin 1:    Bin 2:    Bin 3:    Bin 4:        Bin 5:    Bin 6:
Bin 1:  -1.592534e-02  0.034865  0.025981  0.027086  1.820384e-13  0.032386
Bin 7:  -2.245680e-09  0.032636 -0.006955 -0.029571 -1.113481e-02  0.061881
Bin 13: -4.626566e-09 -0.001186 -0.018218 -0.001950  1.149595e-03  0.114373
Bin 19: -3.881049e-03 -0.025000 -0.038710 -0.015614  2.613844e-02       NaN
Bin 25: -5.894557e-09 -0.032035 -0.013636 -0.017105  5.819263e-02       NaN
Bin 31: -1.071867e-10 -0.023650 -0.013587  0.026046           NaN       NaN


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [64]:
testB = """Bin_1:  -5.766642e-02 -0.099021 -0.136051 -0.130691 -0.162074 -0.299121
Bin_7:  -2.769346e-02 -0.036273 -0.072000 -0.098819 -0.169508 -0.276181
Bin_13: -1.069385e-02 -0.059146 -0.054865 -0.071292 -0.135149 -0.321248
Bin_19: -7.058285e-03 -0.040592 -0.050000 -0.091596 -0.191425       NaN
Bin_25: -6.608292e-09 -0.051372 -0.050000 -0.129273 -0.191433       NaN
Bin_31: -8.101156e-10 -0.046543 -0.059977 -0.138144       NaN       NaN"""
testB = testB.split("\n")

testC = """Bin 1:  -1.592534e-02  0.034865  0.025981  0.027086  1.820384e-13  0.032386
Bin 7:  -2.245680e-09  0.032636 -0.006955 -0.029571 -1.113481e-02  0.061881
Bin 13: -4.626566e-09 -0.001186 -0.018218 -0.001950  1.149595e-03  0.114373
Bin 19: -3.881049e-03 -0.025000 -0.038710 -0.015614  2.613844e-02       NaN
Bin 25: -5.894557e-09 -0.032035 -0.013636 -0.017105  5.819263e-02       NaN
Bin 31: -1.071867e-10 -0.023650 -0.013587  0.026046           NaN       NaN"""
testC = testC.split("\n")

test = testB
table_rows = {}
for row, ii in enumerate(test):
    while("  " in ii):
        ii = str(ii.replace("  ", " "))
    # print(ii.split(" "))
    table_rows[f"Row_{row+1}"] = ii.split(" ")

row_check = 2
col_check = 1
print(f"\nCurrent for Row {row_check} - Column {col_check}: {color.BOLD}{table_rows[f'Row_{row_check}'][col_check]}{color.END}")
ideal = (float(table_rows[f"Row_{row_check-1}"][col_check]) + float(table_rows[f"Row_{row_check+1}"][col_check]))/2
print(f"\t    Calculated Ideal: {color.BBLUE}{ideal}{color.END}\n")


Current for Row 2 - Column 1: -2.769346e-02
	    Calculated Ideal: -0.034180135



In [80]:
import re
import numpy as np
import pandas as pd

ansi_escape = re.compile(r'\x1B\[[0-?]*[ -/]*[@-~]')

def parse_fit_file(path, nrows=6, ncols=6):
    results = []
    current_q2y = None
    current_zpt = None
    parB = None
    parC = None
    chi2 = None
    header_re = re.compile(r"For SMEARED Q2-y Bin (\d+) - z-PT Bin (\d+):")

    with open(path, "r", encoding="utf-8") as f:
        for raw_line in f:
            line = ansi_escape.sub("", raw_line).strip()

            m = header_re.match(line)
            if(m):
                current_q2y = int(m.group(1))
                current_zpt = int(m.group(2))
                parB = None
                parC = None
                chi2 = None

            elif(line.startswith("Par B")):
                try:
                    val = line.split("=")[1].split("±")[0].strip()
                    parB = float(val)
                except Exception:
                    parB = None

            elif(line.startswith("Par C")):
                try:
                    val = line.split("=")[1].split("±")[0].strip()
                    parC = float(val)
                except Exception:
                    parC = None

            elif(line.startswith("chi2/NDF")):
                try:
                    val = line.split("=")[1].strip()
                    chi2 = float(val)
                except Exception:
                    chi2 = None

                if((current_q2y is not None) and (current_zpt is not None)):
                    results.append((current_zpt, parB, parC, chi2))
                parB = None
                parC = None
                chi2 = None

    df = pd.DataFrame(results, columns=["zPT", "ParB", "ParC", "Chi2"])
    if(not df.empty):
        df = df.set_index("zPT").sort_index()

    grid_B = np.full((nrows, ncols), np.nan)
    grid_C = np.full((nrows, ncols), np.nan)

    if(not df.empty):
        for zpt, row in df.iterrows():
            r = (zpt - 1) // ncols
            c = (zpt - 1) % ncols
            grid_B[r, c] = row["ParB"]
            grid_C[r, c] = row["ParC"]

    df_B = pd.DataFrame(grid_B, index=[f"Row {i+1}" for i in range(nrows)],
                        columns=[f"Col {j+1}" for j in range(ncols)])
    df_C = pd.DataFrame(grid_C, index=[f"Row {i+1}" for i in range(nrows)],
                        columns=[f"Col {j+1}" for j in range(ncols)])

    return df, df_B, df_C

# def neighbor_differences(grid):
#     nrows, ncols = grid.shape
#     diff_grid = np.full_like(grid, np.nan, dtype=float)
#     for r in range(nrows):
#         for c in range(ncols):
#             val = grid[r, c]
#             if(np.isnan(val)):
#                 continue
#             neighbors = []
#             # if(r > 0 and not np.isnan(grid[r-1, c])): neighbors.append(grid[r-1, c])
#             # if(r < nrows-1 and not np.isnan(grid[r+1, c])): neighbors.append(grid[r+1, c])
#             if(c > 0 and not np.isnan(grid[r, c-1])):
#                 neighbors.append(grid[r, c-1])
#             else:
#                 neighbors = [val]
#             if(c < ncols-1 and not np.isnan(grid[r, c+1])):
#                 neighbors.append(grid[r, c+1])
#             else:
#                 neighbors = [val]
#             if(len(neighbors) > 0):
#                 avg = np.mean(neighbors)
#                 diff_grid[r, c] = avg# val - avg
#     return pd.DataFrame(diff_grid,
#                         index=[f"Row{i+1}" for i in range(nrows)],
#                         columns=[f"Col{j+1}" for j in range(ncols)])

def neighbor_differences(grid):
    nrows, ncols = grid.shape
    diff_grid = np.full_like(grid, np.nan, dtype=float)

    for r in range(nrows):
        for c in range(ncols):
            val = grid[r, c]
            if(np.isnan(val)):
                continue

            neighbors = []
            if(c > 0 and not np.isnan(grid[r, c-1])):
                neighbors.append(grid[r, c-1])
            if(c < ncols-1 and not np.isnan(grid[r, c+1])):
                neighbors.append(grid[r, c+1])

            if(len(neighbors) == 2):
                avg = np.mean(neighbors)
                diff_grid[r, c] = avg# val - avg
            else:
                # no valid left/right neighbors → keep original value
                diff_grid[r, c] = val

    return pd.DataFrame(diff_grid,
                        index=[f"Row{i+1}" for i in range(nrows)],
                        columns=[f"Col{j+1}" for j in range(ncols)])



df, df_B, df_C = parse_fit_file(filename, nrows=6, ncols=6)



diff_B = neighbor_differences(df_B.values)
diff_C = neighbor_differences(df_C.values)

# a) Fixed number of decimals (example: 4 decimal places)
pd.set_option("display.precision", 5)
# # b) Significant figures (example: 5 sig figs)
# pd.options.display.float_format = "{:.5g}".format
# 2. Add spacing between columns (example: min 12 spaces per column)
pd.set_option("display.colheader_justify", "center")  # optional centering
pd.set_option("display.width", None)  # avoid line wrapping
# When you want to print, use to_string with col_space:
df_B = df_B.to_string(col_space=12)#, float_format="{:.4f}".format)
df_C = df_C.to_string(col_space=12)#, float_format="{:.4f}".format)
diff_B = diff_B.to_string(col_space=12)#, float_format="{:.4f}".format)
diff_C = diff_C.to_string(col_space=12)#, float_format="{:.4f}".format)


print(f"\n{color.BOLD}Grid of Par B:{color.END}\n{df_B}")
print(f"\n\n{color.BOLD}Par B - Neighbor Avg:{color.END}\n{diff_B}")
# print(f"\n{color.BOLD}Grid of Par C:{color.END}\n{df_C}")
# print(f"\n\n{color.BOLD}Par C - Neighbor Avg:{color.END}\n{diff_C}")




Grid of Par B:
                 Col 1        Col 2        Col 3        Col 4        Col 5        Col 6   
Row 1          -0.057666   -0.099021     -0.13605     -0.13069     -0.16207     -0.29912  
Row 2          -0.027693   -0.036273       -0.072    -0.098819     -0.16951     -0.27618  
Row 3          -0.010694   -0.059146    -0.054865    -0.071292     -0.13515     -0.32125  
Row 4         -0.0070583   -0.040592        -0.05    -0.091596     -0.19143          NaN  
Row 5        -6.6083e-09   -0.051372        -0.05     -0.12927     -0.19143          NaN  
Row 6        -8.1012e-10   -0.046543    -0.059977     -0.13814          NaN          NaN  


Par B - Neighbor Avg:
                 Col1         Col2         Col3         Col4         Col5         Col6    
Row1           -0.057666   -0.096859     -0.11486     -0.14906     -0.21491     -0.29912  
Row2           -0.027693   -0.049847    -0.067546     -0.12075      -0.1875     -0.27618  
Row3           -0.010694   -0.032779    -0.065219 

In [ ]:
#splitline{#splitline{Cos(#phi_{h}) Moments vs z}{CLAS12 Pass 2 #color[632]{Preliminary} - (Bayesian) Unfolded with RC}}{#splitline{2.4 < Q^{2} < 2.9}{0.65 < y < 0.75}}

In [82]:
ROOT.kRed

(EColor::kRed) : (unsigned int) 632

In [ ]:
// --- open file and fetch the only object (assumed TCanvas) ---
TFile *f = TFile::Open("Fit_Par_B_RC_Bayesian_Smear_Q2_y_Bin_5_MultiDim_z_pT_Bin_Y_bin_phi_t_VS_Z.root");

if (!f || f->IsZombie()) { Error("macro","Could not open file."); }
TKey *k = (TKey*)f->GetListOfKeys()->At(0);
if (!k) { Error("macro","No keys in file."); }
TCanvas *c = (TCanvas*)k->ReadObj();
if (!c || !c->InheritsFrom(TCanvas::Class())) { Error("macro","First object is not a TCanvas."); }
c->Draw();
c->cd();

// --- enter first sub-pad if one exists ---
TPad *p = nullptr;
{ TIter it(c->GetListOfPrimitives()); TObject *o;
  while ((o = it())) { if (o->InheritsFrom(TPad::Class())) { p = (TPad*)o; break; } }
}
if (p) p->cd();

gPad->Update(); // ensure frame/axes exist

// --- find the multigraph in the current pad ---
TMultiGraph *mg = nullptr;
{ TIter it(gPad->GetListOfPrimitives()); TObject *o;
  while ((o = it())) { if (o->InheritsFrom(TMultiGraph::Class())) { mg = (TMultiGraph*)o; break; } }
}
if (!mg) { Warning("macro","No TMultiGraph found in this pad."); }

// --- set Y range to [-0.45, 0.1] (prefer frame axis if available) ---
if (mg) {
  gPad->Update(); // make sure mg built its frame
  TH1 *frame = mg->GetHistogram(); // created after Draw/Update
  if (frame && frame->GetYaxis()) {
    frame->GetYaxis()->SetRangeUser(-0.45, 0.1);
  } else {
    mg->SetMinimum(-0.45);
    mg->SetMaximum(0.1);
  }
}

// --- set all TGraph* marker sizes inside the multigraph to 2.0 ---
if (mg && mg->GetListOfGraphs()) {
  TIter gIt(mg->GetListOfGraphs()); TObject *obj = nullptr;
  while ((obj = gIt())) {
    if (obj->InheritsFrom(TGraph::Class())) {
      ((TGraph*)obj)->SetMarkerSize(2.0);
    }
  }
}

// --- find a legend, adjust columns, and move to bottom 15% ---
TLegend *leg = nullptr;
{ TIter it2(gPad->GetListOfPrimitives()); TObject *o2;
  while ((o2 = it2())) { if (o2->InheritsFrom(TLegend::Class())) { leg = (TLegend*)o2; break; } }
}
if (leg) {
  leg->SetNColumns(3);
  leg->SetColumnSeparation(0.1); // optional
  leg->SetMargin(0.2);           // optional

  // --- reposition legend to occupy bottom 15% of pad (your chosen numbers) ---
  double x1 = 0.20;     // left margin fraction
  double x2 = 0.90;     // right margin fraction
  double y1 = 0.125;    // small gap from bottom
  double y2 = y1 + 0.1; // height (currently ~10%)

  leg->SetX1NDC(x1);
  leg->SetX2NDC(x2);
  leg->SetY1NDC(y1);
  leg->SetY2NDC(y2);
}

// --- set the title: remove any old TPaveText-based titles, then draw a centered-justified TLatex ---
{
  const char *newTitle =
    "#splitline{#splitline{Cos(#phi_{h}) Moments vs z}"
               "{#font[22]{CLAS12 Pass 2 #color[632]{Preliminary}} - (Bayesian) Unfolded with RC}}"
    "{#splitline{2.4 < Q^{2} < 2.9}{0.65 < y < 0.75}}";

  // Keep internal titles consistent for future redraws
  if (mg) mg->SetTitle(newTitle);
  TH1 *frame = (mg ? mg->GetHistogram() : nullptr);
  if (frame) frame->SetTitle(newTitle);

  // Remove any pre-existing "title" TPaveText boxes
  if (TPaveText *old = (TPaveText*)gPad->GetPrimitive("title")) {
    gPad->GetListOfPrimitives()->Remove(old);
    delete old;
  }
  // (Optional) sweep any other title-like TPaveText objects
  {
    TList *plist = gPad->GetListOfPrimitives();
    if (plist) {
      std::vector<TObject*> toDelete;
      TIter it3(plist); TObject *o3 = nullptr;
      while ((o3 = it3())) {
        if (o3->InheritsFrom(TPaveText::Class())) {
          TString nm = o3->GetName();
          if (nm.BeginsWith("title")) toDelete.push_back(o3);
        }
      }
      for (auto *o : toDelete) { plist->Remove(o); delete o; }
    }
  }

  // Draw the new title at the same position, but with centered justification
  double L = gPad->GetLeftMargin();
  double R = gPad->GetRightMargin();
  double T = gPad->GetTopMargin();
  double x = L + 0.5*(1.0 - L - R);
  double y = 1.0 - 0.5*T;  // keep your previous vertical position

  TLatex *lt = new TLatex();
  lt->SetNDC(true);
  lt->SetTextAlign(23);   // 2 = horizontal center, 3 = vertical top
  lt->SetTextFont(42);
  lt->SetTextSize(0.040);
  lt->DrawLatex(x, y, newTitle);
}


// --- redraw/update ---
gPad->Modified();
gPad->Update();


In [83]:
root_color.Bold

'#font[22]'

In [84]:
ROOT.kBlue

(EColor::kBlue) : (unsigned int) 600

In [ ]:
// --- open file and fetch the only object (assumed TCanvas) ---
TFile *f = TFile::Open("Fit_Par_B_RC_Bayesian_Smear_Q2_y_Bin_5_MultiDim_z_pT_Bin_Y_bin_phi_t_VS_Z.root");



if (!f || f->IsZombie()) { Error("macro","Could not open file."); }
TKey *k = (TKey*)f->GetListOfKeys()->At(0);
if (!k) { Error("macro","No keys in file."); }
TCanvas *c = (TCanvas*)k->ReadObj();
if (!c || !c->InheritsFrom(TCanvas::Class())) { Error("macro","First object is not a TCanvas."); }

// Turn off ROOT's automatic title drawing (prevents TPaveText titles from reappearing)
gStyle->SetOptTitle(0);

c->Draw();
c->cd();

// --- enter first sub-pad if one exists ---
TPad *p = nullptr;
{ TIter it(c->GetListOfPrimitives()); TObject *o;
  while ((o = it())) { if (o->InheritsFrom(TPad::Class())) { p = (TPad*)o; break; } }
}
if (p) p->cd();

gPad->Update(); // ensure frame/axes exist

// --- find the multigraph in the current pad ---
TMultiGraph *mg = nullptr;
{ TIter it(gPad->GetListOfPrimitives()); TObject *o;
  while ((o = it())) { if (o->InheritsFrom(TMultiGraph::Class())) { mg = (TMultiGraph*)o; break; } }
}
if (!mg) { Warning("macro","No TMultiGraph found in this pad."); }

// --- set Y range to [-0.45, 0.1] (prefer frame axis if available) ---
if (mg) {
  gPad->Update(); // make sure mg built its frame
  TH1 *frame = mg->GetHistogram(); // created after Draw/Update
  if (frame && frame->GetYaxis()) {
    frame->GetYaxis()->SetRangeUser(-0.45, 0.1);
  } else {
    mg->SetMinimum(-0.45);
    mg->SetMaximum(0.1);
  }
}



// --- set all TGraph* marker sizes inside the multigraph to 2.0 ---
if (mg && mg->GetListOfGraphs()) {
  TIter gIt(mg->GetListOfGraphs()); TObject *obj = nullptr;
  while ((obj = gIt())) {
    if (obj->InheritsFrom(TGraph::Class())) {
      ((TGraph*)obj)->SetMarkerSize(2.0);
    }
  }
}

// --- find a legend, adjust columns, and move to bottom 15% ---
TLegend *leg = nullptr;
{ TIter it2(gPad->GetListOfPrimitives()); TObject *o2;
  while ((o2 = it2())) { if (o2->InheritsFrom(TLegend::Class())) { leg = (TLegend*)o2; break; } }
}
if (leg) {
  leg->SetNColumns(3);
  leg->SetColumnSeparation(0.1); // optional
  leg->SetMargin(0.2);           // optional

  // --- reposition legend to occupy bottom ~15% of the pad ---
  double x1 = 0.20;     // left margin fraction
  double x2 = 0.90;     // right margin fraction
  double y1 = 0.125;    // small gap from bottom
  double y2 = y1 + 0.1;

  leg->SetX1NDC(x1);
  leg->SetX2NDC(x2);
  leg->SetY1NDC(y1);
  leg->SetY2NDC(y2);
}

// --- remove any pre-existing TPaveText title(s) so we don't get duplicates ---
{
  // common one named exactly "title"
  if (TPaveText *old = (TPaveText*)gPad->GetPrimitive("title")) {
    gPad->GetListOfPrimitives()->Remove(old);
    delete old;
  }
  // also sweep any TPaveText whose name starts with "title"
  if (TList *plist = gPad->GetListOfPrimitives()) {
    std::vector<TObject*> toDelete;
    TIter it3(plist); TObject *o3 = nullptr;
    while ((o3 = it3())) {
      if (o3->InheritsFrom(TPaveText::Class())) {
        TString nm = o3->GetName();
        if (nm.BeginsWith("title")) toDelete.push_back(o3);
      }
    }
    for (auto *o : toDelete) { plist->Remove(o); delete o; }
  }
}

// --- draw the title as 4 centered-justified TLatex lines at the same position ---
{
  // Position: keep your original "top of pad" location
  double Lm = gPad->GetLeftMargin();
  double Rm = gPad->GetRightMargin();
  double Tm = gPad->GetTopMargin();

  const double x = Lm + 0.5*(1.0 - Lm - Rm); // centered in drawable width
  double y = 1.0 - 0.5*Tm;                    // original vertical anchor (near top)

  // Text sizes and spacing (NDC units)
  const double szMain = 0.040;   // main and mid lines
  const double szSub  = 0.036;   // bottom two lines slightly smaller
  const double dyMain = 1.15*szMain; // line spacing factors
  const double dySub  = 1.10*szSub;

  // Create/draw centered lines (SetTextAlign(23) => centered horizontally, top-aligned vertically)
  auto drawLine = [&](const char* name, double yline, double size, const char* txt) {
    if (TObject *prev = gPad->GetPrimitive(name)) { gPad->GetListOfPrimitives()->Remove(prev); delete prev; }
    TLatex *lt = new TLatex();
    lt->SetName(name);
    lt->SetNDC(true);
    lt->SetTextAlign(23);
    lt->SetTextFont(42);
    lt->SetTextSize(size);
    lt->DrawLatex(x, yline, txt);
  };

  // Lines (equivalent to your nested #splitline, but with explicit control + centered justification)
  drawLine("user_title_l1", y,           szMain, "#font[22]{#scale[1.5]{Cos(#phi_{h}) Moments vs z}}");
  y -= 1.5*dyMain;
  drawLine("user_title_l2", y,           szMain, "#font[22]{CLAS12 Pass 2 #color[632]{Preliminary}} - #color[872]{(Bayesian) Unfolded with RC}");
  y -= dyMain;
  drawLine("user_title_l3", y,           szSub,  "#font[22]{2.4 < Q^{2} < 2.9}");
  y -= dySub;
  drawLine("user_title_l4", y,           szSub,  "#font[22]{0.65 < y < 0.75}");
}

List_of_Files.append(["Par_B", "Q2_y_Bin_5", "VS_Z", -0.45, 0.1, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])

// --- final redraw/update ---
gPad->Modified();
gPad->Update();


In [85]:
ROOT.kViolet -8

872

In [58]:
# # ROOT_COMMANDS = """
# # // --- open file and fetch the only object (assumed TCanvas) ---
# # TFile *f = TFile::Open("FILE_NAME_INPUT.root");
# # if (!f || f->IsZombie()) { Error("macro","Could not open file."); }
# # TKey *k = (TKey*)f->GetListOfKeys()->At(0);
# # if (!k) { Error("macro","No keys in file."); }
# # TCanvas *c = (TCanvas*)k->ReadObj();
# # if (!c || !c->InheritsFrom(TCanvas::Class())) { Error("macro","First object is not a TCanvas."); }
# # // Turn off ROOT's automatic title drawing (prevents TPaveText titles from reappearing)
# # gStyle->SetOptTitle(0);
# # c->Draw();
# # c->cd();
# # // --- enter first sub-pad if one exists ---
# # TPad *p = nullptr;
# # { TIter it(c->GetListOfPrimitives()); TObject *o;
# #   while ((o = it())) { if (o->InheritsFrom(TPad::Class())) { p = (TPad*)o; break; } }
# # }
# # if (p) p->cd();
# # gPad->Update(); // ensure frame/axes exist
# # // --- find the multigraph in the current pad ---
# # TMultiGraph *mg = nullptr;
# # { TIter it(gPad->GetListOfPrimitives()); TObject *o;
# #   while ((o = it())) { if (o->InheritsFrom(TMultiGraph::Class())) { mg = (TMultiGraph*)o; break; } }
# # }
# # if (!mg) { Warning("macro","No TMultiGraph found in this pad."); }
# # if (mg) {
# #   gPad->Update(); // make sure mg built its frame
# #   TH1 *frame = mg->GetHistogram(); // created after Draw/Update
# #   if (frame && frame->GetYaxis()) {
# #     frame->GetYaxis()->SetRangeUser(MIN_RANGE, MAX_RANGE);
# #   } else {
# #     mg->SetMinimum(MIN_RANGE);
# #     mg->SetMaximum(MAX_RANGE);
# #   }
# # }
# # // --- set all TGraph* marker sizes inside the multigraph to 2.0 ---
# # if (mg && mg->GetListOfGraphs()) {
# #   TIter gIt(mg->GetListOfGraphs()); TObject *obj = nullptr;
# #   while ((obj = gIt())) {
# #     if (obj->InheritsFrom(TGraph::Class())) {
# #       ((TGraph*)obj)->SetMarkerSize(2.0);
# #     }
# #   }
# # }
# # // --- find a legend, adjust columns, and move to bottom 15% ---
# # TLegend *leg = nullptr;
# # { TIter it2(gPad->GetListOfPrimitives()); TObject *o2;
# #   while ((o2 = it2())) { if (o2->InheritsFrom(TLegend::Class())) { leg = (TLegend*)o2; break; } }
# # }
# # if (leg) {
# #   leg->SetNColumns(3);
# #   leg->SetColumnSeparation(0.1); // optional
# #   leg->SetMargin(0.2);           // optional
# #   // --- reposition legend to occupy bottom ~15% of the pad ---
# #   double x1 = 0.20;     // left margin fraction
# #   double x2 = 0.90;     // right margin fraction
# #   double y1 = 0.125;    // small gap from bottom
# #   double y2 = y1 + 0.1;
# #   leg->SetX1NDC(x1);
# #   leg->SetX2NDC(x2);
# #   leg->SetY1NDC(y1);
# #   leg->SetY2NDC(y2);
# # }
# # // --- remove any pre-existing TPaveText title(s) so we don't get duplicates ---
# # {
# #   // common one named exactly "title"
# #   if (TPaveText *old = (TPaveText*)gPad->GetPrimitive("title")) {
# #     gPad->GetListOfPrimitives()->Remove(old);
# #     delete old;
# #   }
# #   // also sweep any TPaveText whose name starts with "title"
# #   if (TList *plist = gPad->GetListOfPrimitives()) {
# #     std::vector<TObject*> toDelete;
# #     TIter it3(plist); TObject *o3 = nullptr;
# #     while ((o3 = it3())) {
# #       if (o3->InheritsFrom(TPaveText::Class())) {
# #         TString nm = o3->GetName();
# #         if (nm.BeginsWith("title")) toDelete.push_back(o3);
# #       }
# #     }
# #     for (auto *o : toDelete) { plist->Remove(o); delete o; }
# #   }
# # }
# # // --- draw the title as 4 centered-justified TLatex lines at the same position ---
# # {
# #   // Position: keep your original "top of pad" location
# #   double Lm = gPad->GetLeftMargin();
# #   double Rm = gPad->GetRightMargin();
# #   double Tm = gPad->GetTopMargin();
# #   const double x = Lm + 0.5*(1.0 - Lm - Rm); // centered in drawable width
# #   double y = 1.0 - 0.5*Tm;                    // original vertical anchor (near top)
# #   // Text sizes and spacing (NDC units)
# #   const double szMain = 0.040;   // main and mid lines
# #   const double szSub  = 0.036;   // bottom two lines slightly smaller
# #   const double dyMain = 1.15*szMain; // line spacing factors
# #   const double dySub  = 1.10*szSub;
# #   // Create/draw centered lines (SetTextAlign(23) => centered horizontally, top-aligned vertically)
# #   auto drawLine = [&](const char* name, double yline, double size, const char* txt) {
# #     if (TObject *prev = gPad->GetPrimitive(name)) { gPad->GetListOfPrimitives()->Remove(prev); delete prev; }
# #     TLatex *lt = new TLatex();
# #     lt->SetName(name);
# #     lt->SetNDC(true);
# #     lt->SetTextAlign(23);
# #     lt->SetTextFont(42);
# #     lt->SetTextSize(size);
# #     lt->DrawLatex(x, yline, txt);
# #   };
# #   // Lines (equivalent to your nested #splitline, but with explicit control + centered justification)
# #   drawLine("user_title_l1", y,           szMain, "#font[22]{#scale[1.5]{MOMEMENT_TITLE}}");
# #   y -= 1.5*dyMain;
# #   drawLine("user_title_l2", y,           szMain, "#font[22]{CLAS12 Pass 2 #color[632]{Preliminary}} - METHOD_TITLE");
# #   y -= dyMain;
# #   drawLine("user_title_l3", y,           szSub,  "#font[22]{Q2_RANGE}");
# #   y -= dySub;
# #   drawLine("user_title_l4", y,           szSub,  "#font[22]{Y_RANGE}");
# # }
# # // --- final redraw/update ---
# # gPad->Modified();
# # gPad->Update();
# # """

# ROOT_COMMANDS = """// --- open file and fetch the only object (assumed TCanvas) ---
# TFile *f = TFile::Open("FILE_NAME_INPUT.root");

# if (!f || f->IsZombie()) { Error("macro","Could not open file."); }
# TKey *k = (TKey*)f->GetListOfKeys()->At(0);
# if (!k) { Error("macro","No keys in file."); }
# TCanvas *c = (TCanvas*)k->ReadObj();
# if (!c || !c->InheritsFrom(TCanvas::Class())) { Error("macro","First object is not a TCanvas."); }

# // Turn off ROOT's automatic title drawing (prevents TPaveText titles from reappearing)
# gStyle->SetOptTitle(0);

# c->Draw();
# c->cd();

# // --- enter first sub-pad if one exists ---
# TPad *p = nullptr;
# { TIter it(c->GetListOfPrimitives()); TObject *o;
#   while ((o = it())) { if (o->InheritsFrom(TPad::Class())) { p = (TPad*)o; break; } }
# }
# if (p) p->cd();

# gPad->Update(); // ensure frame/axes exist

# // --- find the multigraph in the current pad ---
# TMultiGraph *mg = nullptr;
# { TIter it(gPad->GetListOfPrimitives()); TObject *o;
#   while ((o = it())) { if (o->InheritsFrom(TMultiGraph::Class())) { mg = (TMultiGraph*)o; break; } }
# }
# if (!mg) { Warning("macro","No TMultiGraph found in this pad."); }

# if (mg) {
#   gPad->Update(); // make sure mg built its frame
#   TH1 *frame = mg->GetHistogram(); // created after Draw/Update
#   if (frame && frame->GetYaxis()) {
#     frame->GetYaxis()->SetRangeUser(MIN_RANGE, MAX_RANGE);
#   } else {
#     mg->SetMinimum(MIN_RANGE);
#     mg->SetMaximum(MAX_RANGE);
#   }
# }

# // --- set all TGraph* marker sizes inside the multigraph to 2.5 ---
# if (mg && mg->GetListOfGraphs()) {
#   TIter gIt(mg->GetListOfGraphs()); TObject *obj = nullptr;
#   while ((obj = gIt())) {
#     if (obj->InheritsFrom(TGraph::Class())) {
#       // ((TGraph*)obj)->SetMarkerSize(2.0);
#       ((TGraph*)obj)->SetMarkerSize(2.5);
#     }
#   }
# }

# // --- find a legend, adjust columns, and move to bottom 15% ---
# TLegend *leg = nullptr;
# { TIter it2(gPad->GetListOfPrimitives()); TObject *o2;
#   while ((o2 = it2())) { if (o2->InheritsFrom(TLegend::Class())) { leg = (TLegend*)o2; break; } }
# }
# if (leg) {
#   leg->SetNColumns(3);
#   leg->SetColumnSeparation(0.1); // optional
#   leg->SetMargin(0.2);           // optional

#   // --- reposition legend to occupy bottom ~15% of the pad ---
#   double x1 = 0.20;     // left margin fraction
#   double x2 = 0.90;     // right margin fraction
#   double y1 = 0.125;    // small gap from bottom
#   double y2 = y1 + TLEGEND_HEIGHT;

#   leg->SetX1NDC(x1);
#   leg->SetX2NDC(x2);
#   leg->SetY1NDC(y1);
#   leg->SetY2NDC(y2);
# }

# // --- remove any pre-existing TPaveText title(s) so we don't get duplicates ---
# {
#   // common one named exactly "title"
#   if (TPaveText *old = (TPaveText*)gPad->GetPrimitive("title")) {
#     gPad->GetListOfPrimitives()->Remove(old);
#     delete old;
#   }
#   // also sweep any TPaveText whose name starts with "title"
#   if (TList *plist = gPad->GetListOfPrimitives()) {
#     std::vector<TObject*> toDelete;
#     TIter it3(plist); TObject *o3 = nullptr;
#     while ((o3 = it3())) {
#       if (o3->InheritsFrom(TPaveText::Class())) {
#         TString nm = o3->GetName();
#         if (nm.BeginsWith("title")) toDelete.push_back(o3);
#       }
#     }
#     for (auto *o : toDelete) { plist->Remove(o); delete o; }
#   }
# }

# // --- draw the title as 4 centered-justified TLatex lines at the same position ---
# {
#   // Position: keep your original "top of pad" location
#   double Lm = gPad->GetLeftMargin();
#   double Rm = gPad->GetRightMargin();
#   double Tm = gPad->GetTopMargin();

#   const double x = Lm + 0.5*(1.0 - Lm - Rm); // centered in drawable width
#   double y = 1.0 - 0.5*Tm;                    // original vertical anchor (near top)

#   // Text sizes and spacing (NDC units)
#   const double szMain = 0.040;   // main and mid lines
#   const double szSub  = 0.036;   // bottom two lines slightly smaller
#   const double dyMain = 1.15*szMain; // line spacing factors
#   const double dySub  = 1.10*szSub;

#   // Create/draw centered lines (SetTextAlign(23) => centered horizontally, top-aligned vertically)
#   auto drawLine = [&](const char* name, double yline, double size, const char* txt) {
#     if (TObject *prev = gPad->GetPrimitive(name)) { gPad->GetListOfPrimitives()->Remove(prev); delete prev; }
#     TLatex *lt = new TLatex();
#     lt->SetName(name);
#     lt->SetNDC(true);
#     lt->SetTextAlign(23);
#     lt->SetTextFont(42);
#     lt->SetTextSize(size);
#     lt->DrawLatex(x, yline, txt);
#   };

#   // Lines (equivalent to your nested #splitline, but with explicit control + centered justification)
#   // drawLine("user_title_l1", y,           szMain, "#font[22]{#scale[1.5]{MOMEMENT_TITLE}}");
#   // y -= 1.5*dyMain;
#   drawLine("user_title_l2", y,           szMain, "#font[22]{CLAS12 Pass 2 #color[632]{Preliminary}} - METHOD_TITLE");
#   y -= dyMain;
#   drawLine("user_title_l3", y,           szSub,  "#font[22]{#scale[1.25]{Q2_RANGE}}");
#   y -= 1.25*dySub;
#   drawLine("user_title_l4", y,           szSub,  "#font[22]{#scale[1.25]{Y_RANGE}}");

#   if (mg) {
#     gPad->Update();
#     TH1 *frame = mg->GetHistogram();
#     if (frame && frame->GetXaxis()) {
#       TString xtitle = frame->GetXaxis()->GetTitle();
#       xtitle.ReplaceAll("(Smeared) ", ""); // strip only that part
#       frame->GetXaxis()->SetTitle(xtitle);
#     }
#   }
# }

# // --- final redraw/update ---
# gPad->Modified();
# gPad->Update();
# """

# if(True):
#     print("\n\x1b[91m\x1b[1m// SAVING TCANVASES...\x1b[0m")
#     ROOT_COMMANDS = f"""{ROOT_COMMANDS}
# // Save results:
# c->SaveAs("FILE_NAME_INPUT.pdf");
# // TFile out("FILE_NAME_INPUT.root", "RECREATE"); c->Write(); out.Close();"""

# TLEGEND_HEIGHT = 0.1
# ROOT_COMMANDS = ROOT_COMMANDS.replace("TLEGEND_HEIGHT", TLEGEND_HEIGHT)

# List_of_Files = []

# # # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_5",  "VS_Z", -0.425, 0.05, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_5",  "VS_Z", -0.125, 0.15, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # # List_of_Files.append(["Par_B", "Q2_y_Bin_5",  "VS_Z", -0.45, 0.1, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # # List_of_Files.append(["Par_C", "Q2_y_Bin_5",  "VS_Z", -0.15, 0.2, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_14", "VS_Z", -0.45, 0.1, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# # # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_14", "VS_Z", -0.15, 0.2, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# # # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_5",  "VS_PT", -0.45, 0.1, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_5",  "VS_PT", -0.15, 0.2, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_14", "VS_PT", -0.45, 0.1, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# # # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_14", "VS_PT", -0.15, 0.2, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])


# # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_5",  "VS_Z", -0.575, 0.05, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # List_of_Files.append(["Par_B",    "Q2_y_Bin_5",  "VS_Z", -0.575, 0.05, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_5",  "VS_Z", -0.1,   0.2,  "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # List_of_Files.append(["Par_C",    "Q2_y_Bin_5",  "VS_Z", -0.1,   0.2,  "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])

# # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_14", "VS_Z", -0.575, 0.075, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# # List_of_Files.append(["Par_B",    "Q2_y_Bin_14", "VS_Z", -0.575, 0.075, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_14", "VS_Z", -0.1,   0.15,  "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# # List_of_Files.append(["Par_C",    "Q2_y_Bin_14", "VS_Z", -0.1,   0.15,  "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])

# # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_5",  "VS_PT", -0.575, 0.05, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # List_of_Files.append(["Par_B",    "Q2_y_Bin_5",  "VS_PT", -0.575, 0.05, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_5",  "VS_PT", -0.1,   0.2,  "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # List_of_Files.append(["Par_C",    "Q2_y_Bin_5",  "VS_PT", -0.1,   0.2,  "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])

# # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_14", "VS_PT", -0.575, 0.075, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# # List_of_Files.append(["Par_B",    "Q2_y_Bin_14", "VS_PT", -0.575, 0.075, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# List_of_Files.append(["Par_C_RC", "Q2_y_Bin_14", "VS_PT", -0.1,   0.15,  "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# List_of_Files.append(["Par_C",    "Q2_y_Bin_14", "VS_PT", -0.1,   0.15,  "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])

# output_text = ""
# for par, Bin, var, yMin, yMax, q2_range, y_range in List_of_Files:
#     title_method = "#color[872]{(Bayesian) Unfolded with RC}" if("RC" in par) else "#color[30]{(Bayesian) Unfolded}"
#     title_method = "#color[872]{Unfolded with RC}" if("RC" in par) else "#color[30]{Unfolded without RC}"
#     # title_moment = "Cos(#phi_{h}) Moments " if("Par_B" in par) else "Cos(2#phi_{h}) Moments "
#     # title_moment = f"{title_moment}{'vs z' if('Z' in var) else 'vs P_{T}'}"
#     file_name_in = f"Fit_{par}_Bayesian_Smear_{Bin}_MultiDim_z_pT_Bin_Y_bin_phi_t_{var}"
#     output_text  = f"{output_text}\n{ROOT_COMMANDS}"
#     output_text  = output_text.replace("Q2_RANGE",  q2_range)
#     output_text  = output_text.replace("Y_RANGE",    y_range)
#     output_text  = output_text.replace("MIN_RANGE", str(yMin))
#     output_text  = output_text.replace("MAX_RANGE", str(yMax))
#     output_text  = output_text.replace("METHOD_TITLE",    title_method)
#     # output_text  = output_text.replace("MOMEMENT_TITLE",  title_moment)
#     output_text  = output_text.replace("FILE_NAME_INPUT", file_name_in)
    
# print(f"{output_text}\n\n")


# The Cell Below Can Loop through any bin
The one above is an older version

In [63]:
# start = 3

In [79]:
Save_Q = True
TLEGEND_HEIGHT = 0.1

Unfold_Q = not True # Use Bayesian unfolding if 'True', else show the RC Factors
# Q2_y_Bin_Set_List = [4]
Q2_y_Bin_Set_List = range(1, 18)


# # Q2_y_Bin_Set_List = range(1, 3)
# Q2_y_Bin_Set_List = range(start, start+3)
# start += 3


List_of_Files = []
ROOT_COMMANDS = """// --- open file and fetch the only object (assumed TCanvas) ---
TFile *f = TFile::Open("FILE_NAME_INPUT.root");

if (!f || f->IsZombie()) { Error("macro","Could not open file."); }
TKey *k = (TKey*)f->GetListOfKeys()->At(0);
if (!k) { Error("macro","No keys in file."); }
TCanvas *c = (TCanvas*)k->ReadObj();
if (!c || !c->InheritsFrom(TCanvas::Class())) { Error("macro","First object is not a TCanvas."); }

// Turn off ROOT's automatic title drawing (prevents TPaveText titles from reappearing)
gStyle->SetOptTitle(0);

// c->Draw();
c->cd();

// --- enter first sub-pad if one exists ---
TPad *p = nullptr;
{ TIter it(c->GetListOfPrimitives()); TObject *o;
  while ((o = it())) { if (o->InheritsFrom(TPad::Class())) { p = (TPad*)o; break; } }
}
if (p) p->cd();

gPad->Update(); // ensure frame/axes exist

// --- find the multigraph in the current pad ---
TMultiGraph *mg = nullptr;
{ TIter it(gPad->GetListOfPrimitives()); TObject *o;
  while ((o = it())) { if (o->InheritsFrom(TMultiGraph::Class())) { mg = (TMultiGraph*)o; break; } }
}
if (!mg) { Warning("macro","No TMultiGraph found in this pad."); }

if (mg) {
  gPad->Update(); // make sure mg built its frame
  TH1 *frame = mg->GetHistogram(); // created after Draw/Update
  if (frame && frame->GetYaxis()) {
    frame->GetYaxis()->SetRangeUser(MIN_RANGE, MAX_RANGE);
  } else {
    mg->SetMinimum(MIN_RANGE);
    mg->SetMaximum(MAX_RANGE);
  }
}

// --- set all TGraph* marker sizes inside the multigraph to 2.5 ---
if (mg && mg->GetListOfGraphs()) {
  TIter gIt(mg->GetListOfGraphs()); TObject *obj = nullptr;
  while ((obj = gIt())) {
    if (obj->InheritsFrom(TGraph::Class())) {
      // ((TGraph*)obj)->SetMarkerSize(2.0);
      ((TGraph*)obj)->SetMarkerSize(2.5);
    }
  }
}

// --- find a legend, adjust columns, and move to bottom 15% ---
TLegend *leg = nullptr;
{ TIter it2(gPad->GetListOfPrimitives()); TObject *o2;
  while ((o2 = it2())) { if (o2->InheritsFrom(TLegend::Class())) { leg = (TLegend*)o2; break; } }
}
if (leg) {
  leg->SetNColumns(3);
  leg->SetColumnSeparation(0.1); // optional
  leg->SetMargin(0.2);           // optional

  // --- reposition legend to occupy bottom ~15% of the pad ---
  double x1 = 0.20;     // left margin fraction
  double x2 = 0.90;     // right margin fraction
  double y1 = 0.125;    // small gap from bottom
  double y2 = y1 + TLEGEND_HEIGHT;

  leg->SetX1NDC(x1);
  leg->SetX2NDC(x2);
  leg->SetY1NDC(y1);
  leg->SetY2NDC(y2);
}

// --- remove any pre-existing TPaveText title(s) so we don't get duplicates ---
{
  // common one named exactly "title"
  if (TPaveText *old = (TPaveText*)gPad->GetPrimitive("title")) {
    gPad->GetListOfPrimitives()->Remove(old);
    delete old;
  }
  // also sweep any TPaveText whose name starts with "title"
  if (TList *plist = gPad->GetListOfPrimitives()) {
    std::vector<TObject*> toDelete;
    TIter it3(plist); TObject *o3 = nullptr;
    while ((o3 = it3())) {
      if (o3->InheritsFrom(TPaveText::Class())) {
        TString nm = o3->GetName();
        if (nm.BeginsWith("title")) toDelete.push_back(o3);
      }
    }
    for (auto *o : toDelete) { plist->Remove(o); delete o; }
  }
}

// --- draw the title as 4 centered-justified TLatex lines at the same position ---
{
  // Position: keep your original "top of pad" location
  double Lm = gPad->GetLeftMargin();
  double Rm = gPad->GetRightMargin();
  double Tm = gPad->GetTopMargin();

  const double x = Lm + 0.5*(1.0 - Lm - Rm); // centered in drawable width
  double y = 1.0 - 0.5*Tm;                    // original vertical anchor (near top)

  // Text sizes and spacing (NDC units)
  const double szMain = 0.040;   // main and mid lines
  const double szSub  = 0.036;   // bottom two lines slightly smaller
  const double dyMain = 1.15*szMain; // line spacing factors
  const double dySub  = 1.10*szSub;

  // Create/draw centered lines (SetTextAlign(23) => centered horizontally, top-aligned vertically)
  auto drawLine = [&](const char* name, double yline, double size, const char* txt) {
    if (TObject *prev = gPad->GetPrimitive(name)) { gPad->GetListOfPrimitives()->Remove(prev); delete prev; }
    TLatex *lt = new TLatex();
    lt->SetName(name);
    lt->SetNDC(true);
    lt->SetTextAlign(23);
    lt->SetTextFont(42);
    lt->SetTextSize(size);
    lt->DrawLatex(x, yline, txt);
  };

  // Lines (equivalent to your nested #splitline, but with explicit control + centered justification)
  // drawLine("user_title_l1", y,           szMain, "#font[22]{#scale[1.5]{MOMEMENT_TITLE}}");
  // y -= 1.5*dyMain;
  drawLine("user_title_l2", y,           szMain, "METHOD_TITLE");
  y -= dyMain;
  drawLine("user_title_l3", y,           szSub,  "#font[22]{#scale[1.25]{Q2_RANGE}}");
  y -= 1.25*dySub;
  drawLine("user_title_l4", y,           szSub,  "#font[22]{#scale[1.25]{Y_RANGE}}");

  if (mg) {
    gPad->Update();
    TH1 *frame = mg->GetHistogram();
    if (frame && frame->GetXaxis()) {
      TString xtitle = frame->GetXaxis()->GetTitle();
      xtitle.ReplaceAll("(Smeared) ", ""); // strip only that part
      frame->GetXaxis()->SetTitle(xtitle);
    }
  }
}

// --- final redraw/update ---
gPad->Modified();
gPad->Update();
"""

if(Save_Q):
    print("\n\x1b[91m\x1b[1m// SAVING TCANVASES...\x1b[0m")
    ROOT_COMMANDS = f"""{ROOT_COMMANDS}
// Save results:
c->SaveAs("FILE_NAME_INPUT.pdf");
// TFile out("FILE_NAME_INPUT.root", "RECREATE"); c->Write(); out.Close();"""

ROOT_COMMANDS = ROOT_COMMANDS.replace("TLEGEND_HEIGHT", str(TLEGEND_HEIGHT))


Q2_Bin_Ranges = { "1_Q2": "2.00 < Q^{2} < 2.40",
        		   "1_y":  "0.75 < y < 0.65", 
        		  "2_Q2": "2.00 < Q^{2} < 2.40",
        		   "2_y":  "0.65 < y < 0.55", 
        		  "3_Q2": "2.00 < Q^{2} < 2.40",
        		   "3_y":  "0.55 < y < 0.45", 
        		  "4_Q2": "2.00 < Q^{2} < 2.40",
        		   "4_y":  "0.45 < y < 0.35", 
        		  "5_Q2": "2.40 < Q^{2} < 2.90",
        		   "5_y":  "0.75 < y < 0.65", 
        		  "6_Q2": "2.40 < Q^{2} < 2.90",
        		   "6_y":  "0.65 < y < 0.55", 
        		  "7_Q2": "2.40 < Q^{2} < 2.90",
        		   "7_y":  "0.55 < y < 0.45", 
        		  "8_Q2": "2.40 < Q^{2} < 2.90",
        		   "8_y":  "0.45 < y < 0.35", 
        		  "9_Q2": "2.90 < Q^{2} < 3.70",
        		   "9_y":  "0.75 < y < 0.65", 
        		 "10_Q2": "2.90 < Q^{2} < 3.70",
        		  "10_y":  "0.65 < y < 0.55", 
        		 "11_Q2": "2.90 < Q^{2} < 3.70",
        		  "11_y":  "0.55 < y < 0.45", 
        		 "12_Q2": "2.90 < Q^{2} < 3.70",
        		  "12_y":  "0.45 < y < 0.35", 
        		 "13_Q2": "3.70 < Q^{2} < 5.30",
        		  "13_y":  "0.75 < y < 0.65", 
        		 "14_Q2": "3.70 < Q^{2} < 5.30",
        		  "14_y":  "0.65 < y < 0.55", 
        		 "15_Q2": "3.70 < Q^{2} < 5.30",
        		  "15_y":  "0.55 < y < 0.45", 
        		 "16_Q2": "5.30 < Q^{2} < 7.90",
        		  "16_y":  "0.75 < y < 0.65", 
        		 "17_Q2": "5.30 < Q^{2} < 7.90",
        		  "17_y":  "0.65 < y < 0.55", 
        		 }

for Q2_y_Bin_Set in Q2_y_Bin_Set_List:
    if(Unfold_Q):
        # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_5",  "VS_PT", -0.575, 0.05, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
        # List_of_Files.append(["Par_B",    "Q2_y_Bin_5",  "VS_PT", -0.575, 0.05, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
        # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_5",  "VS_PT", -0.1,   0.2,  "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
        # List_of_Files.append(["Par_C",    "Q2_y_Bin_5",  "VS_PT", -0.1,   0.2,  "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
        
        # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_14", "VS_PT", -0.575, 0.075, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
        # List_of_Files.append(["Par_B",    "Q2_y_Bin_14", "VS_PT", -0.575, 0.075, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
        # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_14", "VS_PT", -0.1,   0.15,  "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
        # List_of_Files.append(["Par_C",    "Q2_y_Bin_14", "VS_PT", -0.1,   0.15,  "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
        
        List_of_Files.append(["Par_B_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.6,    0.2,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        # List_of_Files.append(["Par_B",    f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.6,    0.2,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        # List_of_Files.append(["Par_C_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.45, 0.25, Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        # List_of_Files.append(["Par_C",    f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.45, 0.25, Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
    else:
        List_of_Files.append(["Par_A_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z",  0.825, 1.1,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        # List_of_Files.append(["Par_B_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.2,  0.0,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        # List_of_Files.append(["Par_C_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.04, 0.05, Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        # List_of_Files.append(["Par_C_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.05, 0.05, Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])


output_text = ""
for par, Bin, var, yMin, yMax, q2_range, y_range in List_of_Files:
    if(Unfold_Q):
        title_method = "#color[872]{Unfolded with RC}" if("RC" in par) else "#color[30]{Unfolded without RC}"
        title_method = f"#font[22]{{CLAS12 Pass 2 #color[632]{{Preliminary}}}} - {title_method}"
        file_name_in = f"Fit_{par}_Bayesian_Smear_{Bin}_MultiDim_z_pT_Bin_Y_bin_phi_t_{var}"
    elif("RC" in par):
        title_method = "#color[4]{EvGen RC Factors}"
        file_name_in = f"Fit_{par}_Smear_{Bin}_phi_t_{var}"
    else:
        print("// No 'RC' in 'par' --> Must skip...")
        continue
    output_text  = f"{output_text}\n{ROOT_COMMANDS}"
    output_text  = output_text.replace("Q2_RANGE",  q2_range)
    output_text  = output_text.replace("Y_RANGE",    y_range)
    output_text  = output_text.replace("MIN_RANGE", str(yMin))
    output_text  = output_text.replace("MAX_RANGE", str(yMax))
    output_text  = output_text.replace("METHOD_TITLE",    title_method)
    # output_text  = output_text.replace("MOMEMENT_TITLE",  title_moment)
    output_text  = output_text.replace("FILE_NAME_INPUT", file_name_in)
    
print(f"{output_text}\n\n.q\n")


// SAVING TCANVASES...

// --- open file and fetch the only object (assumed TCanvas) ---
TFile *f = TFile::Open("Fit_Par_A_RC_Smear_Q2_y_Bin_1_phi_t_VS_Z.root");

if (!f || f->IsZombie()) { Error("macro","Could not open file."); }
TKey *k = (TKey*)f->GetListOfKeys()->At(0);
if (!k) { Error("macro","No keys in file."); }
TCanvas *c = (TCanvas*)k->ReadObj();
if (!c || !c->InheritsFrom(TCanvas::Class())) { Error("macro","First object is not a TCanvas."); }

// Turn off ROOT's automatic title drawing (prevents TPaveText titles from reappearing)
gStyle->SetOptTitle(0);

// c->Draw();
c->cd();

// --- enter first sub-pad if one exists ---
TPad *p = nullptr;
{ TIter it(c->GetListOfPrimitives()); TObject *o;
  while ((o = it())) { if (o->InheritsFrom(TPad::Class())) { p = (TPad*)o; break; } }
}
if (p) p->cd();

gPad->Update(); // ensure frame/axes exist

// --- find the multigraph in the current pad ---
TMultiGraph *mg = nullptr;
{ TIter it(gPad->GetListOfPrimitives()); TObject *o;
  while ((

### Extra Lines

In [44]:
test = "Q2_Bin_Ranges = {"
Q2 = [2, 2.4, 2.9, 3.7, 5.3, 7.9]
y_col = 0
Q2row = 0
for ii in range(1, 18):
    test = f"{test}'{ii:>2.0f}_Q2': '{Q2[Q2row]:1.2f} < Q^{{2}} < {Q2[Q2row+1]:1.2f}',\n\t\t  '{ii:>2.0f}_y':  '{0.75-(y_col*0.1):1.2f} < y < {0.65-(y_col*0.1):1.2f}', \n\t\t "
    y_col += 1
    if(ii in [4, 8, 12, 15, 17]):
        y_col  = 0
        Q2row += 1
    
test = test.replace("'", '"')
test = test.replace('" ', ' "')
print(f"\n{test}}}\n")


Q2_Bin_Ranges = { "1_Q2": "2.00 < Q^{2} < 2.40",
		   "1_y":  "0.75 < y < 0.65", 
		  "2_Q2": "2.00 < Q^{2} < 2.40",
		   "2_y":  "0.65 < y < 0.55", 
		  "3_Q2": "2.00 < Q^{2} < 2.40",
		   "3_y":  "0.55 < y < 0.45", 
		  "4_Q2": "2.00 < Q^{2} < 2.40",
		   "4_y":  "0.45 < y < 0.35", 
		  "5_Q2": "2.40 < Q^{2} < 2.90",
		   "5_y":  "0.75 < y < 0.65", 
		  "6_Q2": "2.40 < Q^{2} < 2.90",
		   "6_y":  "0.65 < y < 0.55", 
		  "7_Q2": "2.40 < Q^{2} < 2.90",
		   "7_y":  "0.55 < y < 0.45", 
		  "8_Q2": "2.40 < Q^{2} < 2.90",
		   "8_y":  "0.45 < y < 0.35", 
		  "9_Q2": "2.90 < Q^{2} < 3.70",
		   "9_y":  "0.75 < y < 0.65", 
		 "10_Q2": "2.90 < Q^{2} < 3.70",
		  "10_y":  "0.65 < y < 0.55", 
		 "11_Q2": "2.90 < Q^{2} < 3.70",
		  "11_y":  "0.55 < y < 0.45", 
		 "12_Q2": "2.90 < Q^{2} < 3.70",
		  "12_y":  "0.45 < y < 0.35", 
		 "13_Q2": "3.70 < Q^{2} < 5.30",
		  "13_y":  "0.75 < y < 0.65", 
		 "14_Q2": "3.70 < Q^{2} < 5.30",
		  "14_y":  "0.65 < y < 0.55", 
		 "15_Q2": "3.70 < Q^{2} < 5.30"

In [48]:
root_color.Blue

4

In [1]:
test = """\\begin{figure}[!h]
    \centering
    \\begin{subfigure}{0.3\\textwidth}
        \includegraphics[width=\\textwidth, height=\\textwidth]{All_Images/Analysis_Procedure_Section/Radiative_Corrections/Fit_Par_A_RC_Smear_Q2_y_Bin_#_phi_t_VS_Z.pdf}
        \caption{RC Fit Parameter A}
    \end{subfigure}%
    \\begin{subfigure}{0.3\\textwidth}
        \includegraphics[width=\\textwidth, height=\\textwidth]{All_Images/Analysis_Procedure_Section/Radiative_Corrections/Fit_Par_B_RC_Smear_Q2_y_Bin_#_phi_t_VS_Z.pdf}
        \caption{RC Fit Parameter B}
    \end{subfigure}%
    \\begin{subfigure}{0.3\\textwidth}
        \includegraphics[width=\\textwidth, height=\\textwidth]{All_Images/Analysis_Procedure_Section/Radiative_Corrections/Fit_Par_C_RC_Smear_Q2_y_Bin_#_phi_t_VS_Z.pdf}
        \caption{RC Fit Parameter C}
    \end{subfigure}
    \caption{RC Factor Parameters as functions of $z$ in $Q^2$-$y$ Bin #. See \Cref{FitForMomentsFunction} for definitions.}
    \label{fig:RC_Fit_Factors_for_Q2_y_Bin_#}
\end{figure}

"""

test2 = "\n\n"
for ii in range(1, 18):
    test2 = f"{test2}{test.replace('#', str(ii))}"


print(test2)



\begin{figure}[!h]
    \centering
    \begin{subfigure}{0.3\textwidth}
        \includegraphics[width=\textwidth, height=\textwidth]{All_Images/Analysis_Procedure_Section/Radiative_Corrections/Fit_Par_A_RC_Smear_Q2_y_Bin_1_phi_t_VS_Z.pdf}
        \caption{RC Fit Parameter A}
    \end{subfigure}%
    \begin{subfigure}{0.3\textwidth}
        \includegraphics[width=\textwidth, height=\textwidth]{All_Images/Analysis_Procedure_Section/Radiative_Corrections/Fit_Par_B_RC_Smear_Q2_y_Bin_1_phi_t_VS_Z.pdf}
        \caption{RC Fit Parameter B}
    \end{subfigure}%
    \begin{subfigure}{0.3\textwidth}
        \includegraphics[width=\textwidth, height=\textwidth]{All_Images/Analysis_Procedure_Section/Radiative_Corrections/Fit_Par_C_RC_Smear_Q2_y_Bin_1_phi_t_VS_Z.pdf}
        \caption{RC Fit Parameter C}
    \end{subfigure}
    \caption{RC Factor Parameters as functions of $z$ in $Q^2$-$y$ Bin 1. See \Cref{FitForMomentsFunction} for definitions.}
    \label{fig:RC_Fit_Factors_for_Q2_y_Bin_1}
\end{

In [3]:
test = """\\begin{figure}[h!]
    \centering
    \includegraphics[width=0.95\\textwidth, height=0.4\\textheight]{All_Images/Data_Collection_Section/MC_Simulation_Sections/Pass_2_clasdis_preunfold/Multi_3D_Unfolded_Histos_Q2_y_Bin_#_Data_Smeared.png}
    \caption{\label{fig:Pre_Unfolded_Phi_h_Q2_y_Bin_#}$\phi_{h}$ Comparisons between Data and the \\textsc{clasdis} MC for all events in $Q^2$-$y$ Bin #. See \Cref{fig:Pre_Unfolded_Phi_h_Q2_y_Bin_1} for more details}
\end{figure}

"""

test2 = "\n\n"
for ii in range(1, 18):
    test2 = f"{test2}{test.replace('#', str(ii))}"


print(test2)



\begin{figure}[h!]
    \centering
    \includegraphics[width=0.95\textwidth, height=0.4\textheight]{All_Images/Data_Collection_Section/MC_Simulation_Sections/Pass_2_clasdis_preunfold/Multi_3D_Unfolded_Histos_Q2_y_Bin_1_Data_Smeared.png}
    \caption{\label{fig:Pre_Unfolded_Phi_h_Q2_y_Bin_1}$\phi_{h}$ Comparisons between Data and the \textsc{clasdis} MC for all events in $Q^2$-$y$ Bin 1. See \Cref{fig:Pre_Unfolded_Phi_h_Q2_y_Bin_1} for more details}
\end{figure}

\begin{figure}[h!]
    \centering
    \includegraphics[width=0.95\textwidth, height=0.4\textheight]{All_Images/Data_Collection_Section/MC_Simulation_Sections/Pass_2_clasdis_preunfold/Multi_3D_Unfolded_Histos_Q2_y_Bin_2_Data_Smeared.png}
    \caption{\label{fig:Pre_Unfolded_Phi_h_Q2_y_Bin_2}$\phi_{h}$ Comparisons between Data and the \textsc{clasdis} MC for all events in $Q^2$-$y$ Bin 2. See \Cref{fig:Pre_Unfolded_Phi_h_Q2_y_Bin_1} for more details}
\end{figure}

\begin{figure}[h!]
    \centering
    \includegraphics[width=0.9

In [11]:
len([["y", "$y$"], ["z", "$z$"], ["pT", "$P_T$"], ["xB", "$x_B$"], ["phi_t", "$\phi_h$"], ["W", "$W$"], ["MM", "Missing Mass"], ["el", "Electron Momentum"], ["elth", "$\\theta_{Electron}$"], ["elPhi", "$\phi_{Electron}$"],  ["pip", "$\pi^+$ Pion Momentum"], ["pipth", "$\\theta_{\pi^{+}\\text{ Pion}}$"], ["pipPhi", "$\phi_{\pi^{+}\\text{ Pion}}$"]])

13

In [31]:

# test = """\\begin{figure}[h!]
#     \centering
#     \includegraphics[width=0.85\\textwidth, height=0.275\\textheight, trim=0 400 25 0,clip]{All_Images/Data_Collection_Section/MC_Simulation_Sections/Pass_2_Kinematic_Comparisons_with_EvGen/Kinematic_Comparison_of_VAR_High_Mx_Norm_Full_EvGen_Weighed.png}
#     \caption{\label{fig:Kinematic_Compare_VAR}Comparisons between Data and both MCs of the TEXT_var distributions. See \Cref{fig:Kinematic_Compare_Q2} for more details}
# \end{figure}

# """

test = """\\begin{figure}[h!]
    \centering
    \includegraphics[width=1\\textwidth, trim=0 400 25 0,clip]{All_Images/Data_Collection_Section/MC_Simulation_Sections/Pass_2_Kinematic_Comparisons_with_EvGen/Kinematic_Comparison_of_VAR_High_Mx_Norm_Full_EvGen_Weighed.png}
    \caption{\label{fig:Kinematic_Compare_VAR}Comparisons between Data and both MCs of the TEXT_var distributions. See \Cref{fig:Kinematic_Compare_Q2} for more details}
\end{figure}

"""

test2 = "\n\n"
for ii, jj in [["y", "$y$"], ["z", "$z$"], ["pT", "$P_T$"], ["xB", "$x_B$"], ["phi_t", "$\phi_h$"], ["W", "$W$"], ["MM", "Missing Mass"], ["el", "Electron Momentum"], ["elth", "$\\theta_{Electron}$"], ["elPhi", "$\phi_{Electron}$"],  ["pip", "$\pi^+$ Pion Momentum"], ["pipth", "$\\theta_{\pi^{+}\\text{ Pion}}$"], ["pipPhi", "$\phi_{\pi^{+}\\text{ Pion}}$"]]:
    test2 = f"{test2}{test.replace('VAR', str(ii))}"
    test2 = test2.replace("TEXT_var", str(jj))


print(test2)



\begin{figure}[h!]
    \centering
    \includegraphics[width=1\textwidth, trim=0 400 25 0,clip]{All_Images/Data_Collection_Section/MC_Simulation_Sections/Pass_2_Kinematic_Comparisons_with_EvGen/Kinematic_Comparison_of_y_High_Mx_Norm_Full_EvGen_Weighed.png}
    \caption{\label{fig:Kinematic_Compare_y}Comparisons between Data and both MCs of the $y$ distributions. See \Cref{fig:Kinematic_Compare_Q2} for more details}
\end{figure}

\begin{figure}[h!]
    \centering
    \includegraphics[width=1\textwidth, trim=0 400 25 0,clip]{All_Images/Data_Collection_Section/MC_Simulation_Sections/Pass_2_Kinematic_Comparisons_with_EvGen/Kinematic_Comparison_of_z_High_Mx_Norm_Full_EvGen_Weighed.png}
    \caption{\label{fig:Kinematic_Compare_z}Comparisons between Data and both MCs of the $z$ distributions. See \Cref{fig:Kinematic_Compare_Q2} for more details}
\end{figure}

\begin{figure}[h!]
    \centering
    \includegraphics[width=1\textwidth, trim=0 400 25 0,clip]{All_Images/Data_Collection_Section/M

In [ ]:
TFile *f = TFile::Open("Unfolded_Histos_From_Just_RooUnfold_SIDIS_richcap_Lower_Acceptance_Cut.root");

// Bin ranges (index 0 unused so index = bin number)
const char* Q2_ranges[18] = {
    "", "2.00 < Q^{2} < 2.40", "2.00 < Q^{2} < 2.40", "2.00 < Q^{2} < 2.40", "2.00 < Q^{2} < 2.40",
    "2.40 < Q^{2} < 2.90", "2.40 < Q^{2} < 2.90", "2.40 < Q^{2} < 2.90", "2.40 < Q^{2} < 2.90",
    "2.90 < Q^{2} < 3.70", "2.90 < Q^{2} < 3.70", "2.90 < Q^{2} < 3.70", "2.90 < Q^{2} < 3.70",
    "3.70 < Q^{2} < 5.30", "3.70 < Q^{2} < 5.30", "3.70 < Q^{2} < 5.30",
    "5.30 < Q^{2} < 7.90", "5.30 < Q^{2} < 7.90"
};

const char* y_ranges[18] = {
    "", "0.75 < y < 0.65", "0.65 < y < 0.55", "0.55 < y < 0.45", "0.45 < y < 0.35",
    "0.75 < y < 0.65", "0.65 < y < 0.55", "0.55 < y < 0.45", "0.45 < y < 0.35",
    "0.75 < y < 0.65", "0.65 < y < 0.55", "0.55 < y < 0.45", "0.45 < y < 0.35",
    "0.75 < y < 0.65", "0.65 < y < 0.55", "0.55 < y < 0.45",
    "0.75 < y < 0.65", "0.65 < y < 0.55"
};

// Loop over the 17 histograms
for (int i = 1; i <= 17; i++) {
    TString hname = Form("(Response_Matrix)_(mdf)_(SMEAR=Smear)_(Q2_y_Bin_%d)_(z_pT_Bin_All)_(MultiDim_z_pT_Bin_Y_bin_phi_t)", i);
    TH2D *h = (TH2D*)f->Get(hname);
    if (!h) { printf("Missing: %s\n", hname.Data()); continue; }

    TCanvas *c = new TCanvas(Form("c%d", i), "", 1600, 1400);
    gStyle->SetOptStat(0);
    c->SetLogz();
    h->Draw("colz");

    // Now build the fancier title
    TString title = Form(
        "#splitline{#splitline{Response Matrix of 3D Kinematic Bins (z+P_{T}+#phi_{h})}"
        "{Q^{2}-y Bin: %d #topbar #splitline{%s}{%s}}}{#color[4]{Pass 2}}",
        i, Q2_ranges[i], y_ranges[i]
    );

    h->SetTitle(title);
    h->GetXaxis()->SetTitle("3D Kinematic Bins (z+P_{T}+#phi_{h} - GEN)");
    h->GetYaxis()->SetTitle("3D Kinematic Bins (z+P_{T}+#phi_{h} - REC)");

    TString outname = Form("3D_Response_Matrix_for_Q2_y_Bin_%d_Pass_2_Smeared.pdf", i);
    c->SaveAs(outname);
    delete c;
}
f->Close();


In [8]:
root_color.Blue

4

In [9]:
TFile *f = TFile::Open("Unfolded_Histos_From_Just_RooUnfold_SIDIS_richcap_Lower_Acceptance_Cut.root");
TH2D *h = (TH2D*)f->Get("(Response_Matrix)_(mdf)_(SMEAR=Smear)_(Q2_y_Bin_All)_(z_pT_Bin_All)_(phi_t)");
if (!h) {
    printf("Histogram not found!\n");
} else {
    TCanvas *c = new TCanvas("c_all", "", 800, 700);
    gStyle->SetOptStat(0);
    c->SetLogz();
    h->Draw("colz");
    h->SetTitle("#splitline{Response Matrix of #phi_{h}}{Pass 2 #topbar All Bins}");
    h->GetXaxis()->SetTitle("#phi_{h} (GEN)");
    h->GetYaxis()->SetTitle("#phi_{h} (REC)");
    c->SaveAs("1D_Response_Matrix_for_Q2_y_Bin_All_Pass_2_Smeared.pdf");
    delete c;
}
f->Close();


'#topbar'